<a href="https://colab.research.google.com/github/bethbertozzi/Hate-Speech-Detection-in-Tweets/blob/main/hate_speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Relevant Packages

In [ ]:
import sys
sys.version

'3.6.9 (default, Oct  8 2020, 12:12:24) \n[GCC 8.4.0]'

In [ ]:
import torch
import torchtext
!pip3 install fastai
import fastai
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
import pandas as pd
import torchtext
!pip install pyspellchecker
from spellchecker import SpellChecker
import nltk
from nltk.stem import *
import glob
from fastai.text import *
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import spacy
import urllib.parse

In [ ]:
contractions = { # https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

# Loading Data

In [ ]:
# Mount gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

project_path = '/content/gdrive/My Drive/APS360/APS360: Team Project/Project' 
data_path =  '/content/gdrive/My Drive/APS360/APS360: Team Project/Project/Project Data' 

# For testing
dummy_test_path = (data_path + '/hatespeech' + "/stage2tweets.csv")
dummy_data_set = (pd.read_csv(dummy_test_path, index_col=False)).head(20)

sentiment_data_path = data_path +'/CleanedSentiment_NoIdx.csv'
hatespeech_data_path = data_path + '/CleanedHateDataCombinedNoIdx.csv'

# Sentiment Data
sentiment_data = pd.read_csv(sentiment_data_path) # id, label, tweet

# Hatespeech Data
hatespeech_data = pd.read_csv(hatespeech_data_path)

# For testing
dummy_test_path = (data_path + '/hatespeech' + "/stage2tweets.csv")
dummy_data_set = (pd.read_csv(dummy_test_path, index_col=False)).head(20)

Mounted at /content/gdrive


In [ ]:
def check_balance(ds):
  percentpos = (sum(ds["label"] == 1)/(ds.shape[0]))*100
  percentneg = (sum(ds["label"] == 0)/(ds.shape[0]))*100
  print("Percent of data that is non-negative: {}% | Percent of data that is negative: {}%".format(round(percentpos,3),round(percentneg,3)))

#check_balance(sentiment_data)
#check_balance(hatespeech_data)

**Load Slang Dictionary**

In [ ]:
dict_path = data_path + "/slang_dictionary.csv"
slang = pd.read_csv(dict_path).iloc[:, 0:2]
print(slang.head())
slang_decode = slang.applymap(lambda x: urllib.parse.unquote_plus(x)) # decode url encoding
slang_dict = pd.Series(slang_decode.description.values, index=slang_decode.title.values).to_dict()
print(slang_dict.items())

# Data Pre-Processing

**Remove Usernames, Hashtags**

In [ ]:
def remove_pattern(input_txt, pattern): # INPUT: Text you want to manipulate (str), text you want to remove (str) (Fcn taken from https://www.analyticsvidhya.com/blog/2018/07/hands-on-sentiment-analysis-dataset-python/)
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt # OUTPUT: Cleaned text (str)

def remove_usernames_hashtags(data_set): # INPUT: Pandas dataframe (with usernames, hashtags)
    copy = data_set.copy()
    for i in range(copy.shape[0]):
      copy.loc[i,"tweet"] = remove_pattern(copy.loc[i,"tweet"], "@[\w]*") #Removes any word with the first character "@"
      copy.loc[i,"tweet"] = remove_pattern(copy.loc[i,"tweet"], "#") # Should just removes the hashtag but keep the hashtag word
      copy.loc[i,"tweet"] = re.sub(r"http\S+", "", copy.loc[i,"tweet"])
      copy.loc[i,"tweet"] = remove_pattern(copy.loc[i,"tweet"], "amp")
      if (i%1000 == 0):
        print("Stage 1 is {} % complete.".format((i/copy.shape[0])*100))
    return copy # OUTPUT: New pandas dataframe with usernames removed, hashtags deleted (with usernames, hashtags)

def test_remove_usernames_hashtags():
  dummy = dummy_data_set.copy()
  dummy = dummy.head(10)
  print(dummy)
  print(remove_usernames_hashtags(dummy))

test_remove_usernames_hashtags()

**Replace Slang**

In [ ]:
def slang_lookup(word):
  return slang_dict.get(word.lower(),word) # if word not present in dictionary just return original word

def replace_slang(data_set): # INPUT: Pandas dataframe
  copy = data_set.copy()
  for i in range(copy.shape[0]):
    words = copy.loc[i,"tweet"].split()
    slang_free_words = [slang_lookup(word) for word in words]
    copy.loc[i,"tweet"] = " ".join(slang_free_words) # put back in a string
    if (i%1000 == 0):
      print("Stage 2 is {} % complete.".format((i/copy.shape[0])*100))
  return copy

def test_replace_slang():
  dummy = dummy_data_set.copy()
  slang_dummy = remove_usernames_hashtags(dummy.head(10))
  slang_free_dummy = replace_slang(slang_dummy)
  print(slang_dummy)
  print(slang_free_dummy)

test_replace_slang()

**Remove Short Words, Numbers, Special Characters**

In [ ]:
def replace_words(s, words): # https://stackoverflow.com/questions/20502862/python-replacing-words-in-a-string-with-entries-from-a-dictionary
    for k, v in words.items():
        s = s.replace(k, v)
    return s

def remove_contract(data_set): # INPUT: Pandas dataframe, tweets with contractions
  copy = data_set.copy()
  for i in range(copy.shape[0]): 
    copy.loc[i,"tweet"] = replace_words(copy.loc[i,"tweet"], contractions)
    if (i%1000 == 0):
      print("Stage 3 is {} % complete.".format((i/copy.shape[0])*100))
  return copy # OUTPUT: Pandas dataframe, contractions expanded into separate words

def remove_num_sym(data_set): # INPUT: Pandas dataframe with non-alphabetic characters
  copy = data_set.copy()
  for i in range(copy.shape[0]): 
    copy.loc[i,"tweet"] = re.sub("[^a-zA-Z]+", " ", copy.loc[i,"tweet"])
    if (i%1000 == 0):
      print("Stage 4 is {} % complete.".format((i/copy.shape[0])*100))
  return copy # OUTPUT: New pandas dataframe with no non-alphabetic characters

def remove_short_words(data_set): # INPUT: Pandas dataframe with short words
  copy = data_set.copy()
  for i in range(copy.shape[0]): 
    copy.loc[i,"tweet"] = ' '.join([word for word in copy.loc[i,"tweet"].split() if len(word)>=3]) # Get rid of all words that are less than 2 characters long 
    if (i%1000 == 0):
      print("Stage 5 is {} % complete.".format((i/copy.shape[0])*100))
  return copy # OUTPUT: New pandas dataframe with no words under 3 letters long

def remove_capitalization(data_set): # INPUT: Pandas dataframe with tweets with lower and uppercase letters
  copy = data_set.copy()
  for i in range(copy.shape[0]): 
    copy.loc[i,"tweet"] = copy.loc[i,"tweet"].lower()
    if (i%1000 == 0):
      print("Stage 6 is {} % complete.".format((i/copy.shape[0])*100))
  return copy # OUTPUT: Pandas dataframe with only lowercase tweets

In [ ]:
# testing
def test_remove_contract():
  dummy = dummy_data_set.copy()
  no_contracts_dummy = remove_contract(dummy) 
  print(no_contracts_dummy)

def test_remove_num_sym():
  dummy = dummy_data_set.copy()
  no_num_sym_dummy = remove_num_sym(dummy) 
  print(no_num_sym_dummy)

def test_remove_short_words():
  dummy = dummy_data_set.copy()
  no_short_words_dummy = remove_short_words(dummy) 
  print(no_short_words_dummy)

def test_remove_capitalization():
  dummy = dummy_data_set.copy()
  no_caps_dummy = remove_capitalization(dummy) 
  print(no_caps_dummy)

test_remove_contract()
test_remove_num_sym()
test_remove_short_words()
test_remove_capitalization()

**Correct Spelling**

In [ ]:
def correct(data_set): # INPUT: Pandas dataframe with mispellings
  spell = SpellChecker()
  copy = data_set.copy()
  for i in range(copy.shape[0]): 
    tweet = copy.loc[i,"tweet"].split()
    misspelled = spell.unknown(tweet)
    for word in misspelled:
      copy.loc[i,"tweet"] = re.sub(word, spell.correction(word), copy.loc[i,"tweet"]) 
    if (i%1000==0):
      print("The spellcheck is " + str((i/copy.shape[0])*100) + " complete")
  return copy # OUTPUT: New spellchecked pandas dataframe

def test_correct():
  dummy = dummy_data_set.copy()
  correct_dummy = correct(dummy) 
  print(correct_dummy)

test_correct()

**Stem**


In [ ]:
def stem(data_set):
  porter = PorterStemmer() # use porter algorithm
  copy = data_set.copy()
  for i in range(copy.shape[0]):
    words = copy.loc[i,"tweet"].split()
    stemmed_words = [porter.stem(word) for word in words]
    copy.loc[i,"tweet"] = " ".join(stemmed_words) # put back in a string
  return copy

def test_stem():
  dummy = dummy_data_set.copy()
  stemmed_dummy = stem(dummy)
  print(stemmed_dummy)
test_stem()

**Combine**

In [ ]:
def preprocess(data_set):
  print("Starting:")
  data_set_clean = remove_usernames_hashtags(data_set)
  print("Stage one complete.")
  data_set_clean = replace_slang(data_set_clean)
  print("Stage two complete.")
  data_set_clean = remove_contract(data_set_clean)
  print("Stage three complete.")
  data_set_clean = remove_num_sym(data_set_clean)
  print("Stage four complete.")
  data_set_clean = remove_short_words(data_set_clean)
  print("Stage five complete.")
  data_set_clean = remove_capitalization(data_set_clean)
  print("Stage six complete.")
  data_set_clean = replace_slang(data_set_clean)
  print("Stage seven complete.")
  data_set_clean = correct(data_set_clean)
  print("Stage eight complete.")
  data_set_clean = stem(data_set_clean)
  print("Finished.")
  return data_set_clean

**Process Datasets**

In [ ]:
# Process Sentiment Data
processed_sentiment_data = preprocess(sentiment_data)
processed_sentiment_data.to_csv(processed_sentiment_data_path, index=False)

In [ ]:
# Process Hatespeech Data
processed_hatespeech_data = preprocess(hatespeech_data)
processed_hatespeech_data.to_csv(processed_hatespeech_data_path, index=False)

**Data Analysis for Report:**

In [ ]:
# Show makeup of data:

# Sentiment:
check_balance(processed_sentiment_data)
# Hatespeech:
check_balance(processed_hatespeech_data)

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

objects = ('Non-Negative Tweets', 'Negative Tweets')
y_pos = np.arange(len(objects))
cnt = [sum(processed_sentiment_data["label"] == 1),sum(processed_sentiment_data["label"] == 0)]

plt.bar(y_pos, cnt, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Number of Tweets')
plt.title('Makeup of Sentiment Data')

plt.show()

objects = ('Hate Speech Tweets', 'Non Hate Speech Tweets')
y_pos = np.arange(len(objects))
cnt = [sum(processed_hatespeech_data["label"] == 1),sum(processed_hatespeech_data["label"] == 0)]

plt.bar(y_pos, cnt, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Number of Tweets')
plt.title('Makeup of Hatespeech Data')

plt.show()

In [ ]:
# Show at least one example of cleaned training sample.
print("Uncleaned Tweet: " + sentiment_data.loc[0,"tweet"])
print("Cleaned Tweet: " + processed_sentiment_data.loc[1,"tweet"])

# Reformed Model Process

**Get Data**

In [ ]:
# Testing non-torch method:
glove = torchtext.vocab.GloVe(name="6B", dim=50, max_vectors=1000000) # use 10k most common words

def get_sentiment_data():
    return csv.reader(open(sentiment_data_path,"rt", encoding="latin-1"))

def get_hatespeech_data():
    return csv.reader(open(hatespeech_data_path,"rt", encoding="latin-1"))

.vector_cache/glove.6B.zip: 862MB [06:32, 2.20MB/s]                          
 99%|█████████▉| 396419/400000 [00:10<00:00, 36202.85it/s]

In [ ]:
def split_tweet(tweet):
    # separate punctuations
    tweet = tweet.replace(".", " . ") \
                 .replace(",", " , ") \
                 .replace(";", " ; ") \
                 .replace("?", " ? ")
    return tweet.lower().split()

split_tweet("hello; don't you know?")

['hello', ';', "don't", 'you', 'know', '?']

In [ ]:
def split_dataset(glove_vector, get_data):
    train, valid, test = [], [], []
    for i, line in enumerate(get_data):
        if i==3:
          print(line)
        tweet = line[-1]
        idxs = [glove_vector.stoi[w]        # lookup the index of word
                for w in split_tweet(tweet)
                if w in glove_vector.stoi] # keep words that has an embedding
        if not idxs: # ignore tweets without any word with an embedding
            continue
        idxs = torch.tensor(idxs) # convert list to pytorch tensor
        label = torch.tensor(int(line[0]))
        if (get_data==get_data_sentiment()):
          if i % 2 ==0:
              train.append((idxs, label))
          else:
              valid.append((idxs, label))
        else:
          if i % 5 < 3:
              train.append((idxs, label))
          elif i % 5 == 4:
              valid.append((idxs, label))
          else:
              test.append((idxs, label))
    return train, valid, test

# split datasets into 
train_sentiment, valid_sentiment, placehold = split_dataset(glove, get_sentiment_data())
train_hate, valid_hate, test_hate = split_dataset(glove, get_hatespeech_data())

['1', 'gregabbott tedcruz first day will rescind every illegal executive action taken barack obama gopdebate foxnews']
['0', 'anderson viva based she look like transexual']
38949
tensor(1)
tensor(1)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(1)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)
tensor(0)


In [ ]:
glove_emb = nn.Embedding.from_pretrained(glove.vectors)

**Model**

In [ ]:
class TweetRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(TweetRNN, self).__init__()
        self.name = "TweetRNN"
        self.emb = nn.Embedding.from_pretrained(glove.vectors)
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Look up the embedding
        x = self.emb(x)
        # Set an initial hidden state
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        # Forward propagate the RNN
        out, _ = self.rnn(x, h0)
        # Pass the output of the last time step to the classifier
        out = self.fc(out[:, -1, :])
        return out


**Batching**

In [ ]:
import random

class TweetBatcher:
    def __init__(self, tweets, batch_size=32, drop_last=False):
        # store tweets by length
        self.tweets_by_length = {}
        for words, label in tweets:
            # compute the length of the tweet
            wlen = words.shape[0]
            # put the tweet in the correct key inside self.tweet_by_length
            if wlen not in self.tweets_by_length:
                self.tweets_by_length[wlen] = []
            self.tweets_by_length[wlen].append((words, label),)
         
        #  create a DataLoader for each set of tweets of the same length
        self.loaders = {wlen : torch.utils.data.DataLoader(
                                    tweets,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    drop_last=drop_last) # omit last batch if smaller than batch_size
            for wlen, tweets in self.tweets_by_length.items()}
        
    def __iter__(self): # called by Python to create an iterator
        # make an iterator for every tweet length
        iters = [iter(loader) for loader in self.loaders.values()]
        while iters:
            # pick an iterator (a length)
            im = random.choice(iters)
            try:
                yield next(im)
            except StopIteration:
                # no more elements in the iterator, remove it
                iters.remove(im)

**Training Functions**

In [ ]:
def get_model_name(name, learning_rate, epoch, stage):
    path = "model_{0}_lr{1}_epoch{2}_stage{3}".format(name,learning_rate,epoch,stage)
    return path

In [ ]:
def train_rnn_network(stage, model, train, valid, num_epochs=5, learning_rate=1e-5):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    losses, train_acc, valid_acc = [], [], []
    epochs = []
    for epoch in range(num_epochs):
        for tweets, labels in train:
            optimizer.zero_grad()
            pred = model(tweets)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
        losses.append(float(loss))

        epochs.append(epoch)
        train_acc.append(get_accuracy(model, train))
        valid_acc.append(get_accuracy(model, valid))
        print("Epoch %d; Loss %f; Train Acc %f; Val Acc %f" % (
              epoch+1, loss, train_acc[-1], valid_acc[-1]))
        # Save model checkpoint
        model_path = get_model_name(model.name, learning_rate, num_epochs, stage)
        torch.save(model.state_dict(), model_path)
    # plotting
    plt.title("Training Curve")
    plt.plot(losses, label="Train")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(epochs, train_acc, label="Train")
    plt.plot(epochs, valid_acc, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

**Training**

In [ ]:
# Train Stage 1: Sentiment Classification Model
sent_model = TweetRNN(50, 100, 2)
train_loader_sentiment = TweetBatcher(train_sentiment, batch_size=32, drop_last=True)
valid_loader_sentiment = TweetBatcher(valid_sentiment, batch_size=32, drop_last=True)
train_rnn_network(1, sent_model, train_loader_sentiment, valid_loader_sentiment, num_epochs=15, learning_rate=2e-3)

In [ ]:
# Train Stage 2: Hatespeech Classification Model
hate_model = TweetRNN(50, 100, 2)
train_loader_hate = TweetBatcher(train_hate, batch_size=16, drop_last=False)
valid_loader_hate = TweetBatcher(valid_hate, batch_size=16, drop_last=False)
train_rnn_network(2, hate_model, train_loader_hate, valid_loader_hate, num_epochs=30, learning_rate=2e-4)

# Demonstration

In [ ]:
def OneStageHatePredictor(tweet, hate_model):
  # Process Tweet
  tweet = remove_pattern(tweet, "@[\w]*") #Removes any word with the first character "@"
  tweet = remove_pattern(tweet, "#") # Should just removes the hashtag but keep the hashtag word
  tweet = replace_words(tweet, contractions)
  tweet = re.sub("[^a-zA-Z]+", " ", tweet)
  tweet = ' '.join([word for word in tweet.split() if len(word)>=3])
  tweet = tweet.lower()
  
  # Embed tweet
  glove_vector = glove
  idx = [glove_vector.stoi[w]        # lookup the index of word
          for w in split_tweet(tweet)
          if w in glove_vector.stoi] # keep words that has an embedding
  idx = torch.tensor(idx) # convert list to pytorch tensor

  # Predict 
  pred2 = hate_model(idx.unsqueeze(0))
  max2, indexes = pred2.max(1)
  if max2.item() > 0.5:
    return("Hate")
  else:
    return("Not Hate")

print("This Tweet is...{}".format(HatePredictor("")))

In [ ]:
def OneStageHatePredictorFromTensor(tens, hate_model):
  # Predict 
  pred = sent_model(tens.unsqueeze(0))
  max, indexes = pred.max(1)
  
  if max.item()>1:
    pred2 = hate_model(tens.unsqueeze(0))
    max2, indexes = pred2.max(1)
    if max2.item() > 0.5:
      return 1
    else:
      return 0
  else:
    return 0


NameError: ignored

In [ ]:
def TwoStageHatePredictor(tweet, hate_model, sent_model):
# Process Tweet
  tweet = remove_pattern(tweet, "@[\w]*") #Removes any word with the first character "@"
  tweet = remove_pattern(tweet, "#") # Should just removes the hashtag but keep the hashtag word
  tweet = replace_words(tweet, contractions)
  tweet = re.sub("[^a-zA-Z]+", " ", tweet)
  tweet = ' '.join([word for word in tweet.split() if len(word)>=3])
  tweet = tweet.lower()
  
  # Embed tweet
  glove_vector = glove
  idx = [glove_vector.stoi[w]        # lookup the index of word
          for w in split_tweet(tweet)
          if w in glove_vector.stoi] # keep words that has an embedding
  idx = torch.tensor(idx) # convert list to pytorch tensor

  # Load sentiment model
  sent_model = TweetRNN(50, 10, 2)
  model_path = get_model_name(sent_model.name, 2e-3, 5, 1)
  state = torch.load(model_path)
  sent_model.load_state_dict(state)

  # Load hate model
  hate_model = TweetRNN(50, 100, 2)
  model_path = get_model_name(hate_model.name, 2e-4, 30, 2)
  state = torch.load(model_path) #/content/model_TweetRNN_lr0.002_epoch5_stage1
  hate_model.load_state_dict(state)

  # Predict 
  pred = sent_model(idx.unsqueeze(0))
  max, indexes = pred.max(1)
  if max.item()>1:
    pred2 = hate_model(idx.unsqueeze(0))
    max2, indexes = pred2.max(1)
    if max2.item() > 0.5:
      return("Hate")
    else:
      return("Not Hate")
  else:
    return("Not Hate")

print("This Tweet is...{}".format(TwoStageHatePredictor("")))

In [ ]:
def TwoStageHatePredictorFromTensor(tensor, hate_model, sent_model):
# Load sentiment model
  sent_model = TweetRNN(50, 10, 2)
  model_path = get_model_name(sent_model.name, 2e-3, 5, 1)
  state = torch.load(model_path)
  sent_model.load_state_dict(state)

  # Load hate model
  hate_model = TweetRNN(50, 100, 2)
  model_path = get_model_name(hate_model.name, 2e-4, 30, 2)
  state = torch.load(model_path) #/content/model_TweetRNN_lr0.002_epoch5_stage1
  hate_model.load_state_dict(state)

  # Predict 
  pred = sent_model(tensor.unsqueeze(0))
  max, indexes = pred.max(1)
  if max.item()>1:
    pred2 = hate_model(tensor.unsqueeze(0))
    max2, indexes = pred2.max(1)
    if max2.item() > 0.5:
      return("Hate")
    else:
      return("Not Hate")
  else:
    return("Not Hate")

In [ ]:
def get_accuracy(model, data_loader):
    correct, total = 0, 0
    for tweets, labels in data_loader:
        output = model(tweets)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += labels.shape[0]
    return correct / total

In [ ]:
def get_two_stage_accuracy(data_loader):
    correct, total = 0, 0
    for tweets,labels in data_loader:
      for i in range(len(tweets)):
        pred = HatePredictorFromTensor(tweets[i], hate_model)
        if(pred == int(labels[i])):
          correct +=1
      total += labels.shape[0]
    return correct / total

In [ ]:
# test accuracy from both stages
test_hate_iter = TweetBatcher(test_hate, batch_size=32, drop_last=True)
print(get_two_stage_accuracy(test_hate_iter))

In [ ]:
# test accuracy from only hatespeech model
test_hate_iter = TweetBatcher(test_hate, batch_size=32, drop_last=True)
print(get_accuracy(hate_model, test_hate_iter))

0.8244964454976303


In [ ]:
# Confusion Matrix
false_pos = 0
false_neg = 0
true_neg = 0
true_pos = 0
for tweets,labels in test_hate_iter:
  output = model(tweets)
  pred = output.max(1, keepdim=True)[1]
  for i in range(len(tweets)):
    if (int(pred[i]) is 1) and (int(labels[i]) is 1):
      true_pos+=1
    elif (int(pred[i]) is 1) and (int(labels[i]) is 0):
      false_pos+=1
    elif (int(pred[i]) is 0) and (int(labels[i]) is 1):
      false_neg+=1  
    elif (int(pred[i]) is 0) and (int(labels[i]) is 0):
      true_neg+=1
print("false_pos: " + str(false_pos))
print("false_neg: " + str(false_neg))
print("true_neg: " + str(true_neg))
print("true_pos: " + str(true_pos))

false_pos: 511
false_neg: 716
true_neg: 2992
true_pos: 2533
